In [24]:
#
# hw4pr3titanic:  titanic-passenger clasification via nearest neighbors
#

In [25]:
# libraries!
import numpy as np      # numpy is Python's "array" library
import pandas as pd     # Pandas is Python's "data" library ("dataframe" == spreadsheet)

In [26]:
# let's read in our digits data...
# 
# for read_csv, use header=0 when row 0 is a header row
# 
filename = 'titanic.csv'
df = pd.read_csv(filename, header=0)   # read the file w/header row #0
print(f"{filename} : file read into a pandas dataframe.")

titanic.csv : file read into a pandas dataframe.


In [27]:
#
# Let's see it!
#
pd.set_option('display.max_rows', 10)  # None for no limit; default: 10
pd.set_option('display.min_rows', 10)  # None for no limit; default: 10
# let's view it!
df

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,the original Kaggle dataset! Adapted from here: https://www.kaggle.com/c/titanic
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",NaN
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",NaN
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",NaN
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON",NaN
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN,NaN
1263,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN,NaN
1264,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN,NaN
1265,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN,NaN


In [28]:
#
# develop your model-building based on the previous examples!
#

#
# First, let's show some examples of dropping columns (features) that need to go...
#

In [29]:
#
# this df.info() call will show the different features (columns), their types, and how many non-null there are ...
#

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1267 entries, 0 to 1266
Data columns (total 15 columns):
 #   Column                                                                                Non-Null Count  Dtype  
---  ------                                                                                --------------  -----  
 0   pclass                                                                                1267 non-null   int64  
 1   survived                                                                              1267 non-null   int64  
 2   name                                                                                  1267 non-null   object 
 3   sex                                                                                   1267 non-null   object 
 4   age                                                                                   1004 non-null   float64
 5   sibsp                                                                              

In [30]:
#
# ok!  let's drop columns that are
#      + "cheating"  (they give away the answer and wouldn't be available)
#      + too sparse  (too many NAs)
#

COLUMNS = df.columns       # get column names
last_column = COLUMNS[-1]  # don't want the "url" column...
df_2 = df.drop(columns=[last_column])  # drop by name is ok

# the "boat" column is present when the lifeboat number is known: too much info!
df_2 = df_2.drop('boat', axis=1)     # or, axis = 1 indicates we want to drop a column, not a row

# the "cabin" column has too many missing values...
df_2 = df_2.drop('cabin', axis=1)

# let's see what's there...
df_2.info()  # re-look at the data ...    
# + some will still have too many NA's!
# + some will still be "cheating"
# + some simply won't be worth having

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1267 entries, 0 to 1266
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1267 non-null   int64  
 1   survived   1267 non-null   int64  
 2   name       1267 non-null   object 
 3   sex        1267 non-null   object 
 4   age        1004 non-null   float64
 5   sibsp      1267 non-null   int64  
 6   parch      1267 non-null   int64  
 7   ticket     1267 non-null   object 
 8   fare       1266 non-null   float64
 9   embarked   1265 non-null   object 
 10  body       116 non-null    float64
 11  home.dest  706 non-null    object 
dtypes: float64(3), int64(4), object(5)
memory usage: 118.9+ KB


In [31]:
#
# let's see everything - again
#

pd.set_option('display.max_rows', 10)  # None for no limit; default: 10
pd.set_option('display.min_rows', 10)  # None for no limit; default: 10
# let's view it!
df_2

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,S,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,S,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,S,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,S,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,S,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...
1262,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,C,328.0,NaN
1263,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,C,NaN,NaN
1264,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,C,304.0,NaN
1265,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,C,NaN,NaN


In [32]:
#
# everything needs to be numeric! we'll convert female/male to 1/0
#

def numberize(s):
    """ converts 'male'/'female' to 0/1 """
    if s == 'female': return 1
    else:  return 0                   # or return int(s=='female')

df_2['sex'] = df_2['sex'].apply(numberize)

# can't run this cell twice!   (we've replaced things!)

In [33]:
#
# let's see everything - again
#

pd.set_option('display.max_rows', 10)  # None for no limit; default: 10
pd.set_option('display.min_rows', 10)  # None for no limit; default: 10
# let's view it!
df_2

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",1,29.0000,0,0,24160,211.3375,S,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",0,0.9167,1,2,113781,151.5500,S,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",1,2.0000,1,2,113781,151.5500,S,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",0,30.0000,1,2,113781,151.5500,S,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",1,25.0000,1,2,113781,151.5500,S,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...
1262,3,0,"Zabour, Miss. Hileni",1,14.5000,1,0,2665,14.4542,C,328.0,NaN
1263,3,0,"Zabour, Miss. Thamine",1,NaN,1,0,2665,14.4542,C,NaN,NaN
1264,3,0,"Zakarian, Mr. Mapriededer",0,26.5000,0,0,2656,7.2250,C,304.0,NaN
1265,3,0,"Zakarian, Mr. Ortin",0,27.0000,0,0,2670,7.2250,C,NaN,NaN


In [34]:
#
# develop your model-building based on the previous examples!
#
# remember to
#    + drop more columns!
#    + dropna values!
#    + set up the model, plsit train/test, find the best_k
#    + once you have a baseline... THEN change the weights
#    + to get a sense of the relative importance of each feature...
#

In [35]:
#
# ok!  let's drop columns that are
#      + "cheating"  (they give away the answer and wouldn't be available)
#      + too sparse  (too many NAs)
#


# the "boat" column is present when the lifeboat number is known: too much info!
    # or, axis = 1 indicates we want to drop a column, not a row
df_2 = df_2.drop('body', axis=1) 

# the "cabin" column has too many missing values...
df_2 = df_2.drop('fare', axis=1)
df_2 = df_2.drop('ticket', axis=1)
df_2 = df_2.drop('name', axis=1)
df_2 = df_2.drop('home.dest', axis=1)
df_2 = df_2.drop('embarked', axis=1)# let's see what's there...
df_2.info()  # re-look at the data ...    
# + some will still have too many NA's!
# + some will still be "cheating"
# + some simply won't be worth having

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1267 entries, 0 to 1266
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1267 non-null   int64  
 1   survived  1267 non-null   int64  
 2   sex       1267 non-null   int64  
 3   age       1004 non-null   float64
 4   sibsp     1267 non-null   int64  
 5   parch     1267 non-null   int64  
dtypes: float64(1), int64(5)
memory usage: 59.5 KB


In [38]:
#
# let's keep our column names in variables, for reference
#
COLUMNS = df_2.columns            # "list" of columns
print(f"COLUMNS is {COLUMNS}\n")  
  # It's a "pandas" list, called an Index
  # use it just as a Python list of strings:
print(f"COLUMNS[0] is {COLUMNS[0]}\n")

# let's create a dictionary to look up any column index by name
COL_INDEX = {}
for i, name in enumerate(COLUMNS):
    COL_INDEX[name] = i  # using the name (as key), look up the value (i)
print(f"COL_INDEX is {COL_INDEX}")

COLUMNS is Index(['pclass', 'survived', 'sex', 'age', 'sibsp', 'parch'], dtype='object')

COLUMNS[0] is pclass

COL_INDEX is {'pclass': 0, 'survived': 1, 'sex': 2, 'age': 3, 'sibsp': 4, 'parch': 5}


In [ ]:
# #
# # we can re-weight different features here...
# #

COL_WEIGHT = {              # could be called Feature weight...
     'month':1.0,
     'day':1.0
 }

 for colname in COL_WEIGHT:
    i = COL_INDEX[colname]    # get the column index, i, of the colname
    weight = COL_WEIGHT[colname]  # from the dictionary above
    print("Weighting", colname, "by", weight)   
    # weighting == "multiplying"
    X_all[:,i] *= weight   # multiply by the weight to give this column ("feature")
    
#
# we scramble the data, to give a different TRAIN/TEST split each time...
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_labeled = X_all[indices]              # we apply the _same_ permutation to each!
y_labeled = y_all[indices]              # again...
print(X_labeled)
print(y_labeled)

NUM_ROWS = X_labeled.shape[0]     # the number of labeled rows
TEST_PERCENT = 0.20
TEST_SIZE = int(TEST_PERCENT*NUM_ROWS)   # no harm in rounding down

X_test = X_labeled[:TEST_SIZE]    # first section are for testing
y_test = y_labeled[:TEST_SIZE]

X_train = X_labeled[TEST_SIZE:]   # all the rest are for training
y_train = y_labeled[TEST_SIZE:]

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows" )

from sklearn.neighbors import KNeighborsClassifier

k = 84   # we don't know what k to use, so we guess!  (this will _not_ be a good value)
knn_model = KNeighborsClassifier(n_neighbors=k)       # here, k is the "k" in kNN

# we train the model (it's one line!)
knn_model.fit(X_train, y_train)                              # yay!  trained!
print("Created and trained a knn classifier with k =", k) 

In [ ]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#
NUM_ROWS = X_labeled.shape[0]     # the number of labeled rows
TEST_PERCENT = 0.20
TEST_SIZE = int(TEST_PERCENT*NUM_ROWS)   # no harm in rounding down

X_test = X_labeled[:TEST_SIZE]    # first section are for testing
y_test = y_labeled[:TEST_SIZE]

X_train = X_labeled[TEST_SIZE:]   # all the rest are for training
y_train = y_labeled[TEST_SIZE:]

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows" )

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

k = 84   # we don't know what k to use, so we guess!  (this will _not_ be a good value)
knn_model = KNeighborsClassifier(n_neighbors=k)       # here, k is the "k" in kNN

# we train the model (it's one line!)
knn_model.fit(X_train, y_train)                              # yay!  trained!
print("Created and trained a knn classifier with k =", k)  